# EnTK Seals Pipeline notebook.

This notebook provides a prototypical implementation of the Seal use case, as it is shown in the [Seal Execution Model](https://docs.google.com/document/d/1E79LfwXG1ZJ1fTQiGsDvSggE6BJSDEqAyHKcCxjqgoY/edit?ts=5af5d13e). Each cell of the notebook creates a necesary component of the pipeline definition.

In [ ]:
import os

from radical.entk import Pipeline, Stage, Task, AppManager, ResourceManager

## Pipeline Definition

The next cell defines the prototype pipeline for the Seal Use case. The pipeline has two stages, and each stage has a single task.
The first stage executes the prediction and the second the detection.

What needs to be added is [Stage number 0](https://docs.google.com/document/d/1E79LfwXG1ZJ1fTQiGsDvSggE6BJSDEqAyHKcCxjqgoY/edit?ts=5af5d13e) and the last stage that aggregates the results. Also, the single task in both cases should be broken to multiple tasks based on the number of images.

In [ ]:
def generate_pipeline(name, stages):  #generate the pipeline of prediction and blob detection

    # Create a Pipeline object
    p = Pipeline()
    p.name = name

    for s_cnt in range(stages):


        if(s_cnt==0):
            # Create a Stage object
            s0 = Stage()
            s0.name = 'Stage %s'%s_cnt
            # Create Task 1, training
            t1 = Task()
            t1.name = 'Predictor'
            t1.pre_exec = ['module load psc_path/1.1',
                           'module load slurm/default',
                           'module load intel/17.4',
                           'module load python3',
                           'module load cuda',
                           'mkdir -p classified_images/crabeater',
                           'mkdir -p classified_images/weddel',
                           'mkdir -p classified_images/pack-ice',
                           'mkdir -p classified_images/other',
                           'source <path_to_env>/pytorchCuda/bin/activate'
                          ]
            t1.executable = 'python3'   # Assign executable to the task   
            # Assign arguments for the task executable
            t1.arguments = ['pt_predict.py','-class_names','crabeater','weddel','pack-ice','other']
            t1.link_input_data = ['<path_to_model>/nn_model.pth.tar',
                                  '<path_to_training_set>/nn_images',
                                  '<path_to_test_set>/test_images'
                                  ]
            t1.upload_input_data = ['pt_predict.py','sealnet_nas_scalable.py']
            t1.cpu_reqs = {'processes': 1,'threads_per_process': 1, 'thread_type': 'OpenMP'}
            t1.gpu_reqs = {'processes': 1,'threads_per_process': 1, 'thread_type': 'OpenMP'}
        
            s0.add_tasks(t1)    
            # Add Stage to the Pipeline
            p.add_stages(s0)
        else:
            # Create a Stage object
            s1 = Stage()
            s1.name = 'Stage %s'%s_cnt
            # Create Task 2,
            t2 = Task()
            t2.pre_exec = ['module load psc_path/1.1',
                           'module load slurm/default',
                           'module load intel/17.4',
                           'module load python3',
                           'module load cuda',
                           'module load opencv',
                           'source <path_to_env>/pytorchCuda/bin/activate',
                           'mkdir -p blob_detected'
                         ]
            t2.name = 'Blob_detector'         
            t2.executable = ['python3']   # Assign executable to the task   
            # Assign arguments for the task executable
            t2.arguments = ['blob_detector.py']
            t2.upload_input_data = ['blob_detector.py']
            t2.link_input_data = ['$Pipeline_%s_Stage_%s_Task_%s/classified_images'%(p.uid, s0.uid, t1.uid)]
            t2.download_output_data = ['blob_detected/'] #Download resuting images 
            t2.cpu_reqs = {'processes': 1,'threads_per_process': 1, 'thread_type': 'OpenMP'}
            t2.gpu_reqs = {'processes': 1, 'threads_per_process': 1, 'thread_type': 'OpenMP'}
            s1.add_tasks(t2)
            # Add Stage to the Pipeline
            p.add_stages(s1)

    return p

## Pipeline generation

 The pipeline is define and now we create an object. Although now it only has 2 stages think that the number of stages might change based on specifics of the application. Thus, allowing the number of stages to be an input shows that possible extension


In [ ]:
p = generate_pipeline(name='Pipeline 1', stages=2)

## Resource description and acquisition

We define a dictionary with the following values:
```
{'resource': the resource to execute the pipeline, e.g. 'xsede.bridges' for Bridges,
 'walltime': The amount of time the resources are needed,
 'cpus': Number of CPUs needed,
 'gpus' : Number of GPUs needed,
 'schema' : Way to access the resource without a password. We reccomend gsissh. ,
 'project': Project to charge,
 'queue' : The queue you submit for example GPU-small
    }
```

After the dictionary is created we acquire the resources by creating a `ResourceManager`

---
Instructions how to install gsissh on Ubuntu can be found [here](https://github.com/vivek-bala/docs/blob/master/misc/gsissh_setup_stampede_ubuntu_xenial.sh)

In [ ]:
res_dict = {'resource': 'xsede.bridges',
             'walltime': 30,
             'cpus': 12,
             'gpus' : 2,
             'schema' : 'gsisshh',
             'project': '',
             'queue' : 'GPU-small'
    }
    
# Create Resource Manager
rman = ResourceManager(res_dict)


## Execution

In order to execute the pipeline we create an application manager and assign to it th resource manager previously created. We also assign the generated pipeline.

Finally, we request from the application manager to run the application and we wait for it to finish.

In [ ]:
# Create Application Manager
appman = AppManager(port=32773)

# Assign resource manager to the Application Manager
appman.resource_manager = rman

# Assign the workflow as a set of Pipelines to the Application Manager
appman.assign_workflow(set([p]))

# Run the Application Manager
appman.run()